In [7]:
loaded_data = []
with open("rep1.bed", 'r') as file:
    for line in file:
        # Split the line by tabs
        parts = line.strip().split('\t')
        if len(parts) == 10 and parts[0][3:]!='X':
            # Extract the required data
            #print("**",parts[0][3:],int(parts[0][3:])*1_000_000_000)
            chr_value = int(parts[0][3:])*1_000_000_000
            start_pos = int(parts[1])
            end_pos = int(parts[2])
            feature1 = float(parts[6])
            feature2 = float(parts[7])
            feature3 = float(parts[8])
            feature4 = int(parts[9])

            # Create a data point as a tuple
            data_point = (chr_value, start_pos, end_pos, feature1, feature2, feature3, feature4)
            loaded_data.append(data_point)

In [8]:
loaded_data

[(1000000000, 225474842, 225475404, 482.85414, -1.0, 4.94451, 275),
 (6000000000, 53171796, 53172335, 479.6935, -1.0, 4.94451, 249),
 (18000000000, 47749024, 47749618, 468.12633, -1.0, 4.94451, 294),
 (19000000000, 33777938, 33778481, 466.57779, -1.0, 4.94451, 248),
 (10000000000, 75235848, 75236367, 466.44577, -1.0, 4.94451, 244),
 (14000000000, 21092390, 21092955, 462.07186, -1.0, 4.94451, 284),
 (6000000000, 72412127, 72412659, 460.08104, -1.0, 4.94451, 245),
 (12000000000, 57238984, 57239492, 445.81903, -1.0, 4.94451, 269),
 (7000000000, 151172441, 151172994, 444.79965, -1.0, 4.94451, 280),
 (14000000000, 55051355, 55051842, 442.35151, -1.0, 4.94451, 230),
 (12000000000, 54379588, 54380124, 441.90813, -1.0, 4.94451, 248),
 (10000000000, 6088593, 6089157, 438.78545, -1.0, 4.94451, 285),
 (1000000000, 114346412, 114346936, 438.65469, -1.0, 4.94451, 265),
 (12000000000, 53746544, 53747073, 437.5405, -1.0, 4.94451, 275),
 (17000000000, 41662594, 41663107, 433.30235, -1.0, 4.94451, 274)

In [1]:
import tensorflow as tf
print(tf.__version__)

2.10.0
